## Carga modulo comun

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../../common'))

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from purrfect.dataset import EPKADataset, load_partition,save_partition, create_train_valid_loaders
from torch.utils.data import DataLoader

from purrfect.training import train_model,train_validate
import torch.optim as optim

from purrfect.metrics import MetricAccumulator
from purrfect.active_learning import create_new_partition,create_next_partitions, test_model

from sklearn.model_selection import train_test_split
from purrfect.submission import create_submission

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Definición modelo

In [4]:
# Define the double convolution block
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class ChannelAdder(nn.Module):
    def __init__(self):
        super(ChannelAdder, self).__init__()

    def forward(self, x):
        # Extract the first and second channels
        first_channel = x[:, 0, :, :]  # shape: (batch_size, height, width)
        second_channel = x[:, 1, :, :]  # shape: (batch_size, height, width)

        # Compute the first new channel: sqrt(first_channel^2 + second_channel^2)
        new_channel1 = torch.sqrt(first_channel**2 + second_channel**2)

        # Compute the second new channel: 1/2 * arctan(channel2 / channel1)
        new_channel2 = 0.5 * torch.atan(second_channel / first_channel)

        # Add the new channels to the batch (unsqueeze to maintain dimensions)
        new_channel1 = new_channel1.unsqueeze(1)  # shape: (batch_size, 1, height, width)
        new_channel2 = new_channel2.unsqueeze(1)  # shape: (batch_size, 1, height, width)

        # Concatenate the new channels to the original input
        output = torch.cat((x, new_channel1, new_channel2), dim=1)  # shape: (batch_size, 5, height, width)

        return output

# Define the U-Net model
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        # Channel Adder
        self.channel_adder = ChannelAdder()
        # Encoder
        self.encoder1 = DoubleConv(in_channels+2, 64)
        self.encoder2 = DoubleConv(64, 128)
        self.encoder3 = DoubleConv(128, 256)
        self.encoder4 = DoubleConv(256, 512)

        # Bottleneck
        self.bottleneck = DoubleConv(512, 1024)

        # Decoder
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = DoubleConv(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = DoubleConv(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = DoubleConv(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = DoubleConv(128, 64)

        # Final output layer
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Agregar features
        x = self.channel_adder(x)
        
        # Encoder
        e1 = self.encoder1(x)
        e2 = self.encoder2(F.max_pool2d(e1, 2))
        e3 = self.encoder3(F.max_pool2d(e2, 2))
        e4 = self.encoder4(F.max_pool2d(e3, 2))

        # Bottleneck
        b = self.bottleneck(F.max_pool2d(e4, 2))

        # Decoder with skip connections
        d4 = self.upconv4(b)
        d4 = torch.cat((e4, d4), dim=1)
        d4 = self.decoder4(d4)

        d3 = self.upconv3(d4)
        d3 = torch.cat((e3, d3), dim=1)
        d3 = self.decoder3(d3)

        d2 = self.upconv2(d3)
        d2 = torch.cat((e2, d2), dim=1)
        d2 = self.decoder2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat((e1, d1), dim=1)
        d1 = self.decoder1(d1)

        # Final output
        out = self.final_conv(d1)
        return out

## Creación particion inicial

In [5]:
#Creacion de particiones train y valid
init_partition = load_partition("partition_1.json")
train_partition, val_partition = train_test_split(init_partition, test_size=0.2, random_state=42)
save_partition("partition_1_train.json","partitions",train_partition)
save_partition("partition_1_val.json","partitions",val_partition)

## Carga modelo

In [5]:
#Define model
model = UNet( 3, 1)
model = model.to(DEVICE)

In [6]:

# Define Loss
criterion = torch.nn.L1Loss()
current_partition = 1

In [7]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:11<00:00,  6.58it/s, WMAPE=4.19, DICE=0.207, DPEAKS=119, Loss=0.0151]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:11<00:00,  6.46it/s, WMAPE=5.07, DICE=0.158, DPEAKS=89.5, Loss=0.0152]


Epoch [3/50]


...

Epoch [8/50]


Validate Epoch 8: 100%|██████████| 77/77 [00:11<00:00,  6.57it/s, WMAPE=2.96, DICE=0.0841, DPEAKS=61.8, Loss=0.00887]


Epoch [9/50]


Validate Epoch 9: 100%|██████████| 77/77 [00:11<00:00,  6.55it/s, WMAPE=3.21, DICE=0.0789, DPEAKS=54.3, Loss=0.00914]


early stopping: 3 epochs without improvement
Training complete.


In [8]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)

Validate Epoch test: 100%|██████████| 1941/1941 [04:58<00:00,  6.49it/s, WMAPE=2.96, DICE=0.0947, DPEAKS=66.4, Loss=0.00858]


In [9]:
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_2_val: 100%|██████████| 77/77 [00:11<00:00,  6.61it/s, WMAPE=2.96, DICE=0.0943, DPEAKS=65, Loss=0.00843]


In [10]:
model.load_state_dict(torch.load(best_model_path,weights_only=True))

<All keys matched successfully>

In [11]:
current_partition=2

In [12]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Train Epoch 1:   0%|          | 0/307 [00:00<?, ?it/s]

Validate Epoch 1: 100%|██████████| 77/77 [00:10<00:00,  7.03it/s, WMAPE=3.71, DICE=0.15, DPEAKS=95.2, Loss=0.0113]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:11<00:00,  6.54it/s, WMAPE=8.86, DICE=0.0876, DPEAKS=60.3, Loss=0.023]


...

Epoch [11/50]


Validate Epoch 11: 100%|██████████| 77/77 [00:11<00:00,  6.54it/s, WMAPE=2.72, DICE=0.0544, DPEAKS=41.4, Loss=0.00744]


Epoch [12/50]


Validate Epoch 12: 100%|██████████| 77/77 [00:11<00:00,  6.55it/s, WMAPE=3.79, DICE=0.073, DPEAKS=66.4, Loss=0.0108]


early stopping: 3 epochs without improvement
Training complete.


In [13]:
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_3_val: 100%|██████████| 77/77 [00:12<00:00,  6.23it/s, WMAPE=3.59, DICE=0.0727, DPEAKS=58.2, Loss=0.0109]


In [14]:
current_partition=3
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)

In [15]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:11<00:00,  6.60it/s, WMAPE=2.08, DICE=0.0622, DPEAKS=54.3, Loss=0.00575]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:11<00:00,  6.64it/s, WMAPE=5.84, DICE=0.0619, DPEAKS=53, Loss=0.0147]


Epoch [3/50]


...

Epoch [6/50]


Validate Epoch 6: 100%|██████████| 77/77 [00:11<00:00,  6.63it/s, WMAPE=2.08, DICE=0.0511, DPEAKS=46.3, Loss=0.0058]


Epoch [7/50]


Validate Epoch 7: 100%|██████████| 77/77 [00:11<00:00,  6.59it/s, WMAPE=2.2, DICE=0.0616, DPEAKS=55.9, Loss=0.00639]


early stopping: 3 epochs without improvement
Training complete.


In [16]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_4_val: 100%|██████████| 77/77 [00:12<00:00,  6.26it/s, WMAPE=1.87, DICE=0.0494, DPEAKS=32.6, Loss=0.00539]


In [8]:
current_partition=3
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
model.load_state_dict(torch.load(best_model_path,weights_only=True))
current_partition=4

In [9]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:10<00:00,  7.38it/s, WMAPE=2.36, DICE=0.0575, DPEAKS=67.1, Loss=0.0068]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:10<00:00,  7.48it/s, WMAPE=3.82, DICE=0.0745, DPEAKS=60.2, Loss=0.0112]


Epoch [3/50]


...

Epoch [8/50]


Validate Epoch 8: 100%|██████████| 77/77 [00:09<00:00,  8.06it/s, WMAPE=2.85, DICE=0.0497, DPEAKS=41.4, Loss=0.00785]


Epoch [9/50]


Validate Epoch 9: 100%|██████████| 77/77 [00:09<00:00,  8.09it/s, WMAPE=3.52, DICE=0.0483, DPEAKS=43.2, Loss=0.00928]


early stopping: 3 epochs without improvement
Training complete.


In [10]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_5_val: 100%|██████████| 77/77 [00:09<00:00,  8.13it/s, WMAPE=2.02, DICE=0.0498, DPEAKS=37.8, Loss=0.00563]
